# BigQuery ARIMA+ pipelines

> This notebook shows how to use Vertex AI Pipelines and BigQuery ML pipeline components to train and evaluate a demand forecasting model.

* adapting from [this](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_bqml_pipeline_demand_forecasting.ipynb) github tutorial
* link to [colab](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/pipelines/google_cloud_pipeline_components_bqml_pipeline_demand_forecasting.ipynb#scrollTo=P0Ks1UZpoRXS)

steps:
* Define a custom evaluation component
* Define a pipeline:
> * Get BigQuery training data
> * Train a BigQuery Arima Plus model
> * Evaluate the BigQuery Arima Plus model
> * Plot the evaluations
> * Check the model performance
> * Generate the ARIMA Plus forecasts
> * Generate the ARIMA PLUS forecast explainations
* Compile the pipeline.
* Execute the pipeline.

In [1]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1'
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [2]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.19
google_cloud_pipeline_components version: 1.0.39


In [3]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
REGION = 'us-central1'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"REGION: {REGION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
REGION: us-central1


## setup

### UUID (optional)

> If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [4]:
import random
import string

# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

UUID = generate_uuid()
UUID

'fj89gje6'

### create GCS bucket

In [5]:
BUCKET_NAME = f"retail-oracle-adapted-{UUID}"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [12]:
! gsutil mb -p $PROJECT_ID -l $REGION $BUCKET_URI

Creating gs://retail-oracle-adapted-fj89gje6/...


In [13]:
! gsutil ls -al $BUCKET_URI

### import packages

In [14]:
from pathlib import Path as path
from typing import NamedTuple
# General
from urllib.parse import urlparse

import google.cloud.aiplatform as vertex_ai

from google.cloud import bigquery

# ML pipeline
from google_cloud_pipeline_components.v1.bigquery import (
    BigqueryCreateModelJobOp, BigqueryEvaluateModelJobOp,
    BigqueryExplainForecastModelJobOp, BigqueryForecastModelJobOp,
    BigqueryMLArimaEvaluateJobOp, BigqueryQueryJobOp
)

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import HTML, Artifact, Condition, Input, Output, component

### initialize Vertex SDK & BQ client

In [15]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [16]:
bq_client = bigquery.Client(project=PROJECT_ID, location=REGION)

## Create local directories

In [18]:
# !pwd

In [19]:
DATA_PATH = "data"
KFP_COMPONENTS_PATH = "components"
PIPELINES_PATH = "pipelines"

! mkdir -m 777 -p {DATA_PATH}
! mkdir -m 777 -p {KFP_COMPONENTS_PATH}
! mkdir -m 777 -p {PIPELINES_PATH}

## Prepare the training data

In [20]:
PUBLIC_DATA_URI = "gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/oracle_retail/orders.csv"
RAW_DATA_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/orders.csv"

! gsutil -m cp -R $PUBLIC_DATA_URI $RAW_DATA_URI

Copying gs://cloud-samples-data/vertex-ai/pipeline-deployment/datasets/oracle_retail/orders.csv [Content-Type=text/csv]...
- [1/1 files][ 41.0 MiB/ 41.0 MiB] 100% Done                                    
Operation completed over 1 objects/41.0 MiB.                                     


In [21]:
! gsutil cat {RAW_DATA_URI} | head

time_of_sale,order_id,product_name,price,quantity,payment_method,store_id,user_id
2021-11-30 16:00:00,754824,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,2831356,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,3032217,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,4956836,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,7710387,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,7477642,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,15878299,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,7742864,Limes,7.950597391,10,Card:AMEX,231,1012780
2021-11-30 16:00:00,9905111,Limes,7.950597391,10,Card:AMEX,231,1012780


### load to BigQuery

In [22]:
LOCATION = REGION.split('-')[0]

! bq mk --location={LOCATION} --dataset {PROJECT_ID}:fast_fresh 

! bq load \
  --location={LOCATION} \
  --source_format=CSV \
  --skip_leading_rows=1\
  fast_fresh.orders_{UUID} \
  {RAW_DATA_URI} \
  time_of_sale:DATETIME,order_id:INTEGER,product_name:STRING,price:NUMERIC,quantity:NUMERIC,payment_method:STRING,store_id:INTEGER,user_id:INTEGER

Dataset 'hybrid-vertex:fast_fresh' successfully created.
Waiting on bqjob_r5e80603f7f45222d_00000187130eb6ab_1 ... (4s) Current status: DONE   


## BigQuery ML Training Formalization

In the next cells, you build the components and pipeline to train and evaluate the BQML demand forecasting model.

### define pipeline variables

Set up some variables specific for the pipeline.

In [23]:
# BQML pipeline job configuation
PIPELINE_NAME = "bqml-forecast-pipeline"
PIPELINE_ROOT = urlparse(BUCKET_URI)._replace(path="pipeline_root").geturl()
PIPELINE_PACKAGE = str(path(PIPELINES_PATH) / f"{PIPELINE_NAME}.json")

In [24]:
# BQML pipeline conponent configuration
BQ_DATASET = "fast_fresh"
BQ_ORDERS_TABLE_PREFIX = "orders"
BQ_TRAINING_TABLE_PREFIX = "orders_training"
BQ_MODEL_TABLE_PREFIX = "orders_forecast_arima"
BQ_EVALUATE_TS_TABLE_PREFIX = "orders_arima_time_series_evaluate"
BQ_EVALUATE_MODEL_TABLE_PREFIX = "orders_arima_model_evaluate"
BQ_FORECAST_TABLE_PREFIX = "orders_arima_forecast"
BQ_EXPLAIN_FORECAST_TABLE_PREFIX = "orders_arima_explain_forecast"
BQ_ORDERS_TABLE = f"{BQ_ORDERS_TABLE_PREFIX}_{UUID}"
BQ_TRAINING_TABLE = f"{BQ_TRAINING_TABLE_PREFIX}_{UUID}"
BQ_MODEL_TABLE = f"{BQ_MODEL_TABLE_PREFIX}_{UUID}"
BQ_EVALUATE_TS_TABLE = f"{BQ_EVALUATE_TS_TABLE_PREFIX}_{UUID}"
BQ_EVALUATE_MODEL_TABLE = f"{BQ_EVALUATE_MODEL_TABLE_PREFIX}_{UUID}"
BQ_FORECAST_TABLE = f"{BQ_FORECAST_TABLE_PREFIX}_{UUID}"
BQ_EXPLAIN_FORECAST_TABLE = f"{BQ_EXPLAIN_FORECAST_TABLE_PREFIX}_{UUID}"

In [25]:
BQ_TRAIN_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_TRAINING_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}

BQ_EVALUATE_TS_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_EVALUATE_TS_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
BQ_EVALUATE_MODEL_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_EVALUATE_MODEL_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
BQ_FORECAST_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_FORECAST_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
BQ_EXPLAIN_FORECAST_CONFIGURATION = {
    "destinationTable": {
        "projectId": PROJECT_ID,
        "datasetId": BQ_DATASET,
        "tableId": BQ_EXPLAIN_FORECAST_TABLE,
    },
    "writeDisposition": "WRITE_TRUNCATE",
}
PERF_THRESHOLD = 3000

### Create location for storing compoment definitions

Next, you create a Storage Cloud location for storing the YAML component definitions for the custom components you create in this tutorial.

In [26]:
! mkdir -m 777 -p {KFP_COMPONENTS_PATH}/custom_components

### Create a custom component to read model evaluation metrics

Build a custom component to consume model evaluation metrics for visualizations in the Vertex AI Pipelines UI using Kubeflow SDK visualization APIs. Indeed, Vertex AI allows you to render that HTML in an output page which is easily accessible from the Google Cloud console.

In [27]:
@component(
    base_image="python:3.8-slim",
    packages_to_install=["jinja2", "pandas", "matplotlib"],
    output_component_file=f"{KFP_COMPONENTS_PATH}/custom_components/build_bq_evaluate_metrics.yaml",
)
def get_model_evaluation_metrics(
    metrics_in: Input[Artifact], metrics_out: Output[HTML]
) -> NamedTuple("Outputs", [("avg_mean_absolute_error", float)]):
    """
    Get the average mean absolute error from the metrics
    Args:
        metrics_in: metrics artifact
        metrics_out: metrics artifact
    Returns:
        avg_mean_absolute_error: average mean absolute error
    """

    import pandas as pd

    # Helpers
    def prettyfier(styler):
        """
        Helper function to prettify the metrics table.
        Args:
            styler: Styler object
        Returns:
            Styler object
        """
        caption = {
            "selector": "caption",
            "props": [
                ("caption-side", "top"),
                ("font-size", "150%"),
                ("font-weight", "bold"),
                ("font-family", "arial"),
            ],
        }
        headers = {
            "selector": "th",
            "props": [("color", "black"), ("font-family", "arial")],
        }
        rows = {
            "selector": "td",
            "props": [("text-align", "center"), ("font-family", "arial")],
        }
        styler.set_table_styles([caption, headers, rows])
        styler.set_caption("Forecasting accuracy report <br><br>")
        styler.hide(axis="index")
        styler.format(precision=2)
        styler.background_gradient(cmap="Blues")
        return styler

    def get_column_names(header):
        """
        Helper function to get the column names from the metrics table.
        Args:
            header: header
        Returns:
            column_names: column names
        """
        header_clean = header.replace("_", " ")
        header_abbrev = "".join([h[0].upper() for h in header_clean.split()])
        header_prettied = f"{header_clean} ({header_abbrev})"
        return header_prettied

    # Extract rows and schema from metrics artifact
    rows = metrics_in.metadata["rows"]
    schema = metrics_in.metadata["schema"]

    # Convert into a tabular format
    columns = [metrics["name"] for metrics in schema["fields"] if "name" in metrics]
    records = []
    for row in rows:
        records.append([dl["v"] for dl in row["f"]])
    metrics = (
        pd.DataFrame.from_records(records, columns=columns, index="product_name")
        .astype(float)
        .round(3)
    )
    metrics = metrics.reset_index()

    # Create the HTML artifact for the metrics
    pretty_columns = list(
        map(
            lambda h: get_column_names(h)
            if h != columns[0]
            else h.replace("_", " ").capitalize(),
            columns,
        )
    )
    pretty_metrics = metrics.copy()
    pretty_metrics.columns = pretty_columns
    html_metrics = pretty_metrics.style.pipe(prettyfier).to_html()
    with open(metrics_out.path, "w") as f:
        f.write(html_metrics)

    # Create metrics dictionary for the model
    avg_mean_absolute_error = round(float(metrics.mean_absolute_error.mean()), 0)
    component_outputs = NamedTuple("Outputs", [("avg_mean_absolute_error", float)])

    return component_outputs(avg_mean_absolute_error)

### Build the BigQuery ML training pipeline

Define your workflow using Kubeflow Pipelines DSL package. 

Below you have the steps of the pipeline workflow:

1. Get BigQuery training data
2. Train a BigQuery Arima Plus model
3. Evaluate the BigQuery Arima Plus model
4. Plot the evaluations
5. Check the model performance
6. Generate the ARIMA Plus forecasts
7. Generate the ARIMA PLUS forecast explainations

In [28]:
@dsl.pipeline(
    name=PIPELINE_NAME,
    description="A batch pipeline to train ARIMA PLUS using BQML",
)
def pipeline(
    bq_dataset: str = BQ_DATASET,
    bq_orders_table: str = BQ_ORDERS_TABLE,
    bq_training_table: str = BQ_TRAINING_TABLE,
    bq_train_configuration: dict = BQ_TRAIN_CONFIGURATION,
    bq_model_table: str = BQ_MODEL_TABLE,
    bq_evaluate_time_series_configuration: dict = BQ_EVALUATE_TS_CONFIGURATION,
    bq_evaluate_model_configuration: dict = BQ_EVALUATE_MODEL_CONFIGURATION,
    performance_threshold: float = PERF_THRESHOLD,
    bq_forecast_configuration: dict = BQ_FORECAST_CONFIGURATION,
    bq_explain_forecast_configuration: dict = BQ_EXPLAIN_FORECAST_CONFIGURATION,
    project: str = PROJECT_ID,
    location: str = LOCATION,
):

    # ======================================
    # Create the training dataset
    # ======================================
    create_training_dataset_op = BigqueryQueryJobOp(
        query=f"""
        -- create the training table
        WITH 
        -- get 90% percentile for time series splitting
        get_split AS (
          SELECT APPROX_QUANTILES(DATETIME_TRUNC(time_of_sale, HOUR), 100)[OFFSET(90)] as split
          FROM `{project}.{bq_dataset}.{bq_orders_table}`
        ),
        -- get train table
        get_train AS (
          SELECT
            DATETIME_TRUNC(time_of_sale, HOUR) as hourly_timestamp,
            product_name,
            SUM(quantity) AS total_sold,
            FROM `{project}.{bq_dataset}.{bq_orders_table}`
        GROUP BY hourly_timestamp, product_name
        )
        SELECT
          *,
          CASE WHEN hourly_timestamp < (SELECT split FROM get_split) THEN 'TRAIN' ELSE 'TEST' END AS split
        FROM get_train
        ORDER BY hourly_timestamp
        """,
        job_configuration_query=bq_train_configuration,
        project=project,
        location=location,
    ).set_display_name("get train data")

    # ======================================
    # Run an ARIMA PLUS experiment
    # ======================================
    bq_arima_model_exp_op = (
        BigqueryCreateModelJobOp(
            query=f"""
        -- create model table
        CREATE OR REPLACE MODEL `{project}.{bq_dataset}.{bq_model_table}`
        OPTIONS(
        MODEL_TYPE = \'ARIMA_PLUS\',
        TIME_SERIES_TIMESTAMP_COL = \'hourly_timestamp\',
        TIME_SERIES_DATA_COL = \'total_sold\',
        TIME_SERIES_ID_COL = [\'product_name\']
        ) AS
        SELECT
          hourly_timestamp,
          product_name,
          total_sold
        FROM `{project}.{bq_dataset}.{bq_training_table}`
        WHERE split='TRAIN';
        """,
            project=project,
            location=location,
        )
        .set_display_name("run arima+ model experiment")
        .after(create_training_dataset_op)
    )

    # ======================================
    # Evaluate ARIMA PLUS time series
    # ======================================
    _ = (
        BigqueryMLArimaEvaluateJobOp(
            project=project,
            location=location,
            model=bq_arima_model_exp_op.outputs["model"],
            show_all_candidate_models=False,
            job_configuration_query=bq_evaluate_time_series_configuration,
        )
        .set_display_name("evaluate arima plus time series")
        .after(bq_arima_model_exp_op)
    )

    # ======================================
    # Evaluate ARIMA Plus model
    # ======================================
    bq_arima_evaluate_model_op = (
        BigqueryEvaluateModelJobOp(
            project=project,
            location=location,
            model=bq_arima_model_exp_op.outputs["model"],
            query_statement=f"""SELECT * FROM `{project}.{bq_dataset}.{bq_training_table}` WHERE split='TEST'""",
            job_configuration_query=bq_evaluate_model_configuration,
        )
        .set_display_name("evaluate arima plus model")
        .after(bq_arima_model_exp_op)
    )

    # ======================================
    # Plot model metrics
    # ======================================
    get_evaluation_model_metrics_op = (
        get_model_evaluation_metrics(
            bq_arima_evaluate_model_op.outputs["evaluation_metrics"]
        )
        .after(bq_arima_evaluate_model_op)
        .set_display_name("plot evaluation metrics")
    )

    # ======================================
    # Check the model performance: 
    # If ARIMA_PLUS average MAE metric is below to a minimal threshold
    # ======================================
    with Condition(
        get_evaluation_model_metrics_op.outputs["avg_mean_absolute_error"]
        < PERF_THRESHOLD,
        name="avg. mae good",
    ):
        # Train the ARIMA PLUS model
        bq_arima_model_op = (
            BigqueryCreateModelJobOp(
                query=f"""
        -- create model table
        CREATE OR REPLACE MODEL `{project}.{bq_dataset}.{bq_model_table}`
        OPTIONS(
        MODEL_TYPE = \'ARIMA_PLUS\',
        TIME_SERIES_TIMESTAMP_COL = \'hourly_timestamp\',
        TIME_SERIES_DATA_COL = \'total_sold\',
        TIME_SERIES_ID_COL = [\'product_name\'],
        MODEL_REGISTRY = \'vertex_ai\',
        VERTEX_AI_MODEL_ID = \'order_demand_forecasting\',
        VERTEX_AI_MODEL_VERSION_ALIASES = [\'staging\']
        ) AS
        SELECT
          DATETIME_TRUNC(time_of_sale, HOUR) as hourly_timestamp,
          product_name,
          SUM(quantity) AS total_sold,
          FROM `{project}.{bq_dataset}.{bq_orders_table}`
        GROUP BY hourly_timestamp, product_name;
        """,
                project=project,
                location=location,
            )
            .set_display_name("train arima+ model")
            .after(get_evaluation_model_metrics_op)
        )

        # ======================================
        # Generate the ARIMA PLUS forecasts
        # ======================================
        bq_arima_forecast_op = (
            BigqueryForecastModelJobOp(
                project=project,
                location=location,
                model=bq_arima_model_op.outputs["model"],
                horizon=1,  # 1 hour
                confidence_level=0.9,
                job_configuration_query=bq_forecast_configuration,
            )
            .set_display_name("generate hourly forecasts")
            .after(get_evaluation_model_metrics_op)
        )

        # ======================================
        # Generate the ARIMA PLUS forecast explainations
        # ======================================
        _ = (
            BigqueryExplainForecastModelJobOp(
                project=project,
                location=location,
                model=bq_arima_model_op.outputs["model"],
                horizon=1,  # 1 hour
                confidence_level=0.9,
                job_configuration_query=bq_explain_forecast_configuration,
            )
            .set_display_name("explain hourly forecasts")
            .after(bq_arima_forecast_op)
        )

### compile pipeline

In [29]:
compiler.Compiler().compile(
    pipeline_func=pipeline, 
    package_path=PIPELINE_PACKAGE
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


## Execute pipeline

pipeline parameters, which we set as default:

- `bq_dataset`: The BigQuery dataset to train on.
- `bq_orders_table` : The BigQuery table of raw data.
- `bq_training_table` : The BigQuery table of training data after preprocessing.
- `bq_train_configuration` : The job configuration of training component
- `bq_model_table`: The BigQuery table of trained model.
- `bq_evaluate_time_series_configuration` : The job configuration of the evaluation of ARIMA time series
- `bq_evaluate_model_configuration` : The job configuration of the evaluation of ARIMA model
- `performance_threshold` : The value of average MAE threshold 
- `bq_forecast_configuration` : The job configuration of forecast component
- `bq_explain_forecast_configuration` : The job configuration of the evaluation of forecast component
- `project`: The project ID
- `location`: The location

In [30]:
SERVICE_ACCOUNT = 'notebooksa@hybrid-vertex.iam.gserviceaccount.com'

In [31]:
bqml_pipeline = vertex_ai.PipelineJob(
    display_name=f"{PIPELINE_NAME}-job",
    template_path=PIPELINE_PACKAGE,
    pipeline_root=PIPELINE_ROOT,
    failure_policy='fast',
    enable_caching=False,
)

bqml_pipeline.run(
    sync=False,
    service_account=SERVICE_ACCOUNT,
    # network=f'projects/{PROJECT_NUM}/global/networks/{vpc_network_name}'
)

Creating PipelineJob
PipelineJob created. Resource name: projects/934903580331/locations/us-central1/pipelineJobs/bqml-forecast-pipeline-20230324100905
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/934903580331/locations/us-central1/pipelineJobs/bqml-forecast-pipeline-20230324100905')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/bqml-forecast-pipeline-20230324100905?project=934903580331
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/bqml-forecast-pipeline-20230324100905 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/bqml-forecast-pipeline-20230324100905 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/bqml-forecast-pipeline-20230324100905 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/93490358

# Cleanin up resources

In [ ]:
# delete pipeline
vertex_ai_pipeline_jobs = vertex_ai.PipelineJob.list(
    filter=f'pipeline_name="{PIPELINE_NAME}"'
)
for pipeline_job in vertex_ai_pipeline_jobs:
    pipeline_job.delete()

In [ ]:
# delete model
DELETE_MODEL_SQL = f"DROP MODEL {BQ_DATASET}.{BQ_MODEL_TABLE}"
try:
    delete_model_query_job = bq_client.query(DELETE_MODEL_SQL)
    delete_model_query_result = delete_model_query_job.result()
except Exception as e:
    print(e)

In [ ]:
# delete dataset
try:
    delete_detaset_query_result = bq_client.delete_dataset(
        BQ_DATASET, delete_contents=True, not_found_ok=True
    )
    print(delete_detaset_query_result)
except Exception as e:
    print(e)

In [ ]:
# delete bucket
delete_bucket = True
if os.getenv("IS_TESTING") or delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI

In [ ]:
# Remove local resorces
! rm -rf {KFP_COMPONENTS_PATH}
! rm -rf {PIPELINES_PATH}
! rm -rf {DATA_PATH}